# DR8 Calibration Files

The goal of this notebook is to document how we ran [DR8a](https://desi.lbl.gov/trac/wiki/DecamLegacy/DR8#Testregions), including calibration files, with the updated data model and using the burst buffer on Cori.

John Moustakas  
Siena College  
2019 March 10

Many thanks to Dustin Lang, Adam Myers, Eddie Schlafly, David Schlegel, Martin Landriau, and Stephen Bailey.

## A. Preliminaries

### A1. Define $LEGACY_SURVEY_DIR.

First, choose a new, empty top-level directory on `project` and create it:
```bash
export LEGACY_SURVEY_DIR=/global/project/projectdirs/cosmo/work/legacysurvey/dr8a
mkdir -p $LEGACY_SURVEY_DIR
cd $LEGACY_SURVEY_DIR
```
Note: the same `$LEGACY_SURVEY_DIR` environment variable will need to be defined within the scripts below so that they are self-contained.

**Hereafter I will assume that all scripts are launched from the $LEGACY_SURVEY_DIR directory.**

Next, create soft links to the CP-reduced imaging data.
```bash
mkdir -p images
ln -s /global/project/projectdirs/cosmo/staging/90prime images/90prime
ln -s /global/project/projectdirs/cosmo/staging/mosaic images/mosaic
ln -s /global/project/projectdirs/cosmo/staging/decam images/decam
```

### A2. Access (or create) the burst buffer.

For DR8a, Dustin Lang created a 40TB burst-buffer reservation called "DR8".  You can think of this as a mounted external drive, where large files can be written and read without the significant overhead associated with `$SCRATCH`, although eventually the results will be copied onto `project`, as described below.

To access this file system for the first time you have to create a configuration file (just once).
```bash
echo "#DW persistentdw name=DR8" > bb.conf
```

The files in the burst buffer can be accessed *only* from an interactive Cori node.  Let's do this (just once) and make a dedicated subdirectory to keep our outputs tidy:
```bash
salloc -N 1 -C haswell -q interactive -t 00:10:00 --bbf=bb.conf
mkdir -p $DW_PERSISTENT_STRIPED_DR8/dr8a
```
Note that the `$DW_PERSISTENT_STRIPED_DR8` environment variable must be used *always*, as every user will have a different absolute path.

For the record, a new reservation can be made (if necessary) by submitting the following SLURM script to the queue:

```bash
#! /bin/bash
#SBATCH -q debug
#SBATCH -N 1
#SBATCH -C haswell
#SBATCH -t 00:05:00
#BB create_persistent name=BBNAME capacity=50000GB access_mode=striped type=scratch
```
where `BBNAME` is the desired name of the reservation.

### A3. Set up software dependencies.
**ToDo**: Add Docker instructions (link to Adam's NB).  
**ToDo**: Add instructions for compiling tractor and astrometry.net, if needed (separate notebook).  
**ToDo**: QDO may be added to the desiconda stack.

Most of the code dependencies are in the `desiconda` imaging stack (like [tractor](https://github.com/dstndstn/tractor) and [astrometry.net](https://github.com/dstndstn/astrometry.net)), which we will source in our setup script, below, but we usually depend on a recent (hopefully tagged) version of [legacyzpts](https://github.com/legacysurvey/legacyzpts) and [legacypipe](https://github.com/legacysurvey/legacypipe).

```bash
cd $LEGACY_SURVEY_DIR
mkdir -p dr8code
for package in legacyzpts legacypipe; do
  git clone git@github.com:legacysurvey/$package.git dr8code/$package
  cd $package ; git checkout tags/dr8 ; cd ..
done
git clone https://bitbucket.org/berkeleylab/qdo/src/master dr8code/qdo
```

Next, create a bash script called `dr8-env.sh` that will set up all our dependencies and myriad environment variables.  This script isn't checked in because it contains a *qdo* database password, but it should look something like this (with the appropriate `LEGACY_SURVEY_DIR` environment variable, of course):

```bash
#! /bin/bash
export LEGACY_SURVEY_DIR=/global/project/projectdirs/cosmo/work/legacysurvey/dr8a

# Use local check-outs of qdo, legacypipe, and legacyzpts.
export QDO_DIR=$LEGACY_SURVEY_DIR/dr8code/qdo 
export LEGACYPIPE_DIR=$LEGACY_SURVEY_DIR/dr8code/legacypipe
export LEGACYZPTS_DIR=$LEGACY_SURVEY_DIR/dr8code/legacyzpts

# This file has most of the needed environment variables.
source $LEGACYPIPE_DIR/bin/legacypipe-env

export PYTHONPATH=$QDO_DIR:$PYTHONPATH
export PYTHONPATH=$LEGACYPIPE_DIR/py:$PYTHONPATH
export PYTHONPATH=$LEGACYZPTS_DIR/py:$PYTHONPATH

export PATH=$QDO_DIR/bin:$PATH
export PATH=$LEGACYPIPE_DIR/bin:$PATH
export PATH=$LEGACYZPTS_DIR/bin:$PATH

export QDO_BACKEND=postgres
export QDO_BATCH_PROFILE=cori
export QDO_DB_HOST=nerscdb03.nersc.gov
export QDO_DB_NAME=desirun
export QDO_DB_PASS=ask_a_friend_for_this
export QDO_DB_USER=desirun_admin

# Some NERSC-specific options to get MPI working properly.
export OMP_NUM_THREADS=1
export MKL_NUM_THREADS=1
export KMP_AFFINITY=disabled
export MPICH_GNI_FORK_MODE=FULLCOPY
```

### A4. Create the input image lists.

For DR8 Eddie Schlafly and David Schlegel inspected all the `DECam`, `90prime`, and `mosaic` imaging data on-disk and created three FITS files which should be used to define the input data:
* dr8-ondisk-decam.fits
* dr8-ondisk-90prime.fits
* dr8-ondisk-mosaic.fits

We use these tables (cutting on `QKEEP==True`) to create our input image lists.  To keep everything tidy, create a new `$LEGACY_SURVEY_DIR/image-lists` subdirectory, copy the `dr8-ondisk-*.fits` files there, source the `dr8-env.sh` file, and then run the following snippet of code in an `ipython` session:
```python
import os
import fitsio
for camera in ('decam', 'mosaic', '90prime'):
    data = fitsio.read('dr8-ondisk-{}.fits'.format(camera), upper=True)
    with open('image-list-{}.txt'.format(camera), 'w') as flist:
        for imfile in data[data['QKEEP']]['FILENAME']:
            flist.write('{}\n'.format(os.path.join(camera, imfile.decode('utf-8').strip())))
```

### Launch the everything.

### Rsync everything to `project`.

```bash
cd $DW_PERSISTENT_SCRATCH/dr8a
rsync -auv zpts $LEGACY_SURVEY_DIR >> rsync-zpts.log 2>&1 &
rsync -auvR calib/*/*-merged $LEGACY_SURVEY_DIR >> rsync-calib.log 2>&1 &
```

### Build the merged *survey-ccds* and *annotated-ccds* files (and their KD-tree cousins).

```bash
./dr8-merge-zpts.sh
```
Might have to put this in a `slurm` script.  This also generates the KD-tree version of these files.

### Optional: update ccd_cuts and apply depth cuts.

### Run legacypipe

```bash
cd $DW_PERSISTENT_SCRATCH/dr8a/runbrick
rsync -auv tractor* $LEGACY_SURVEY_DIR >> rsync-tractor.log 2>&1 &
rsync -auv coadd $LEGACY_SURVEY_DIR >> rsync-coadd.log 2>&1 &
rsync -auv metrics $LEGACY_SURVEY_DIR >> rsync-metrics.log 2>&1 &
```

```bash
qdo launch dr8a 256 --cores_per_worker 8 --walltime=00:30:00 \
  --script ./dr8-runbrick.sh --batchqueue debug --keep_env --batchopts "--bbf=bb.conf"
```


### Update the viewer

```bash
cd /global/project/projectdirs/cosmo/webapp/viewer-dev/
```
edit load-layer.py and run it!  Takes a long time.  Then “touch wsgi.py” and hit reload